# 파일 다운로드

In [ ]:
pip install google-cloud-vision

In [ ]:
pip install google-cloud-storage

In [ ]:
pip install Flask

# 플라스크에서 이미지를 받아와서 GCS에 업로드 - q

In [ ]:
import os
import base64
import requests
from google.cloud import storage, vision
from google.oauth2 import service_account
import json
import re
from flask import Flask, request, jsonify, send_file

app = Flask(__name__)
app.config['DEBUG'] = False

# API 키 설정 및 클라이언트 초기화
credentials = service_account.Credentials.from_service_account_file('api_key4.json')
vision_client = vision.ImageAnnotatorClient(credentials=credentials)
storage_client = storage.Client(credentials=credentials)

def download_image_from_gcs(bucket_name, source_blob_name, destination_file_name):
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)
    print(f"Blob {source_blob_name} downloaded to {destination_file_name}.")

def extract_expiry_date(image_path):
    with open(image_path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)
    response = vision_client.text_detection(image=image)
    texts = response.text_annotations

    # 오류 처리
    if not texts:
        raise KeyError("응답에 'textAnnotations' 키가 없습니다.")

    # 추출된 텍스트 중에서 유통기한 정보 추출 (예: 정규표현식 사용)
    expiry_date = None

    # 날짜 패턴 정규 표현식 (예: YYYY-MM-DD, YYYY/MM/DD, YYYY.MM.DD)
    date_patterns = [
        r'\d{4}-\d{2}-\d{2}',  # YYYY-MM-DD
        r'\d{4}/\d{2}/\d{2}',  # YYYY/MM/DD
        r'\d{4}\.\d{2}\.\d{2}',  # YYYY.MM.DD
        r'\d{2}-\d{2}-\d{2}',  # YY-MM-DD
        r'\d{2}/\d{2}/\d{2}',  # YY/MM/DD
        r'\d{2}\.\d{2}\.\d{2}'  # YY.MM.DD
    ]

    for text in texts:
        description = text.description
        for pattern in date_patterns:
            match = re.search(pattern, description)
            if match:
                expiry_date = match.group()
                break
        if expiry_date:
            break

    return expiry_date
    
def save_output_to_gcs(bucket_name, destination_blob_name, output_data):
    """구글 스토리지에 출력 값을 저장합니다."""
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    # 출력 값을 문자열로 변환하여 저장합니다.
    output_str = str(output_data)

    # 객체에 데이터를 씁니다.
    blob.upload_from_string(output_str)

    print(f"Output 저장 완료: gs://{bucket_name}/{destination_blob_name}")

@app.route('/process_image', methods=['POST'])
def process_image():
    data = request.json
    bucket_name = data['bucket_name']
    source_blob_name = data['source_blob_name']
    destination_file_name = 'temp_image.jpg'
    destination_blob_name = data['destination_blob_name']

    try:
        download_image_from_gcs(bucket_name, source_blob_name, destination_file_name)
        expiry_date = extract_expiry_date(destination_file_name)
        save_output_to_gcs(bucket_name, destination_blob_name, {"expiry_date": expiry_date})
        return jsonify({"status": "success", "expiry_date": expiry_date})
    except KeyError as e:
        app.logger.error(f"Key error: {e}", exc_info=True)
        return jsonify({"status": "error", "message": f"Key error: {e}"}), 400
    except Exception as e:
        app.logger.error(f"Error: {e}", exc_info=True)
        return jsonify({"status": "error", "message": f"Error: {e}"}), 500

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.219.50:5000
Press CTRL+C to quit


Blob 4.jpg downloaded to temp_image.jpg.


127.0.0.1 - - [17/Jun/2024 09:13:10] "POST /process_image HTTP/1.1" 200 -


Output 저장 완료: gs://plzbe/output.txt


# GCS에 있는 이미지를 OCR시켜서 날짜 데이터를 반환하게 하는 코드 - c

In [ ]:
import os
import base64
import requests
from google.cloud import storage, vision
from google.oauth2 import service_account
import json
import re
from flask import Flask, request, jsonify

app = Flask(__name__)
app.config['DEBUG'] = False

# API 키 설정 및 클라이언트 초기화
credentials = service_account.Credentials.from_service_account_file('api_key4.json')
vision_client = vision.ImageAnnotatorClient(credentials=credentials)
storage_client = storage.Client(credentials=credentials)

def download_image_from_gcs(bucket_name, source_blob_name, destination_file_name):
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)
    print(f"Blob {source_blob_name} downloaded to {destination_file_name}.")

def extract_expiry_date(image_path):
    with open(image_path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)
    response = vision_client.text_detection(image=image)
    texts = response.text_annotations

    # 오류 처리
    if not texts:
        raise KeyError("응답에 'textAnnotations' 키가 없습니다.")

    # 추출된 텍스트 중에서 유통기한 정보 추출 (예: 정규표현식 사용)
    expiry_date = None

    # 날짜 패턴 정규 표현식 (예: YYYY-MM-DD, YYYY/MM/DD, YYYY.MM.DD)
    date_patterns = [
        r'\d{4}-\d{2}-\d{2}',  # YYYY-MM-DD
        r'\d{4}/\d{2}/\d{2}',  # YYYY/MM/DD
        r'\d{4}\.\d{2}\.\d{2}',  # YYYY.MM.DD
        r'\d{2}-\d{2}-\d{2}',  # YY-MM-DD
        r'\d{2}/\d{2}/\d{2}',  # YY/MM/DD
        r'\d{2}\.\d{2}\.\d{2}'  # YY.MM.DD
    ]

    for text in texts:
        descr롤대남주거iption = text.description
        for pattern in date_patterns:
            match = re.search(pattern, description)
            if match:
                expiry_date = match.group()
                break
        if expiry_date:
            break

    return expiry_date
    
def save_output_to_gcs(bucket_name, destination_blob_name, output_data):
    """구글 스토리지에 출력 값을 저장합니다."""
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    # 출력 값을 문자열로 변환하여 저장합니다.
    output_str = str(output_data)

    # 객체에 데이터를 씁니다.
    blob.upload_from_string(output_str)

    print(f"Output 저장 완료: gs://{bucket_name}/{destination_blob_name}")

@app.route('/process_image', methods=['POST'])
def process_image():
    data = request.json
    bucket_name = data['bucket_name']
    source_blob_name = data['source_blob_name']
    destination_file_name = 'temp_image.jpg'
    destination_blob_name = data['destination_blob_name']

    try:
        download_image_from_gcs(bucket_name, source_blob_name, destination_file_name)
        expiry_date = extract_expiry_date(destination_file_name)
        save_output_to_gcs(bucket_name, destination_blob_name, {"expiry_date": expiry_date})
        return jsonify({"status": "success", "expiry_date": expiry_date})
    except KeyError as e:
        app.logger.error(f"Key error: {e}", exc_info=True)
        return jsonify({"status": "error", "message": f"Key error: {e}"}), 400
    except Exception as e:
        app.logger.error(f"Error: {e}", exc_info=True)
        return jsonify({"status": "error", "message": f"Error: {e}"}), 500

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.219.50:5000
Press CTRL+C to quit


# 두 개 합치기 - f

In [43]:
import os
import re
from datetime import datetime
from google.cloud import storage, vision
from google.oauth2 import service_account
from flask import Flask, request, jsonify

app = Flask(__name__)

credentials = service_account.Credentials.from_service_account_file('api_key4.json')
vision_client = vision.ImageAnnotatorClient(credentials=credentials)
storage_client = storage.Client(credentials=credentials)

def download_image_from_gcs(bucket_name, source_blob_name, destination_file_name):
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)
    print(f"Blob {source_blob_name} downloaded to {destination_file_name}.")

def extract_expiry_date(image_path):
    with open(image_path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)
    response = vision_client.text_detection(image=image)
    texts = response.text_annotations

    if not texts:
        raise KeyError("No text found in image.")

    # 날짜 패턴 정규 표현식 (예: YYYY-MM-DD, YYYY/MM/DD, YYYY.MM.DD)
    date_patterns = [
        r'\d{4}-\d{2}-\d{2}',  # YYYY-MM-DD
        r'\d{4}/\d{2}/\d{2}',  # YYYY/MM/DD
        r'\d{4}\.\d{2}\.\d{2}',  # YYYY.MM.DD
        r'\d{2}-\d{2}-\d{2}',  # YY-MM-DD
        r'\d{2}/\d{2}/\d{2}',  # YY/MM/DD
        r'\d{2}\.\d{2}\.\d{2}'  # YY.MM.DD
    ]

    dates = []
    for text in texts:
        description = text.description
        for pattern in date_patterns:
            matches = re.findall(pattern, description)
            for match in matches:
                try:
                    # 날짜 형식을 datetime 객체로 변환
                    if '-' in match:
                        date_obj = datetime.strptime(match, '%Y-%m-%d')
                    elif '/' in match:
                        date_obj = datetime.strptime(match, '%Y/%m/%d')
                    elif '.' in match:
                        date_obj = datetime.strptime(match, '%Y.%m.%d')
                    else:
                        # YY-MM-DD, YY/MM/DD, YY.MM.DD 형식 처리
                        if '-' in match:
                            date_obj = datetime.strptime(match, '%y-%m-%d')
                        elif '/' in match:
                            date_obj = datetime.strptime(match, '%y/%m/%d')
                        elif '.' in match:
                            date_obj = datetime.strptime(match, '%y.%m.%d')
                    dates.append(date_obj)
                except ValueError as e:
                    print(f"Date parsing error: {e}")
                    continue

    if dates:
        latest_date = max(dates)
        print(f"Extracted dates: {dates}")
        print(f"Latest date: {latest_date}")
        return latest_date.strftime('%y.%m.%d')
    return None

def save_output_to_gcs(bucket_name, destination_blob_name, output_data):
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    output_str = str(output_data)
    blob.upload_from_string(output_str)
    print(f"Output saved to gs://{bucket_name}/{destination_blob_name}")

@app.route('/process_image', methods=['POST'])
def process_image():
    data = request.json
    bucket_name = data['bucket_name']
    source_blob_name = data['source_blob_name']
    destination_file_name = 'temp_image.jpg'
    destination_blob_name = data['destination_blob_name']

    try:
        download_image_from_gcs(bucket_name, source_blob_name, destination_file_name)
        expiry_date = extract_expiry_date(destination_file_name)
        save_output_to_gcs(bucket_name, destination_blob_name, {"expiry_date": expiry_date})
        return jsonify({"status": "success", "expiry_date": expiry_date})
    except KeyError as e:
        app.logger.error(f"Key error: {e}", exc_info=True)
        return jsonify({"status": "error", "message": f"Key error: {e}"}), 400
    except Exception as e:
        app.logger.error(f"Error: {e}", exc_info=True)
        return jsonify({"status": "error", "message": f"Error: {e}"}), 500

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


FileNotFoundError: [Errno 2] No such file or directory: 'api_key4.json'

# 가장 느린 값을 출력하도록 변경 - f

In [1]:
import os
import re
from datetime import datetime
from google.cloud import storage, vision
from google.oauth2 import service_account
from flask import Flask, request, jsonify

app = Flask(__name__)

credentials = service_account.Credentials.from_service_account_file('api_key3.json')
vision_client = vision.ImageAnnotatorClient(credentials=credentials)
storage_client = storage.Client(credentials=credentials)

def download_image_from_gcs(bucket_name, source_blob_name, destination_file_name):
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)
    print(f"Blob {source_blob_name} downloaded to {destination_file_name}.")

def extract_expiry_date(image_path):
    with open(image_path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)
    response = vision_client.text_detection(image=image)
    texts = response.text_annotations

    if not texts:
        raise KeyError("No text found in image.")

    # 날짜 패턴 정규 표현식 (예: YYYY-MM-DD, YYYY/MM/DD, YYYY.MM.DD)
    date_patterns = [
        r'\d{4}-\d{2}-\d{2}',  # YYYY-MM-DD
        r'\d{4}/\d{2}/\d{2}',  # YYYY/MM/DD
        r'\d{4}\.\d{2}\.\d{2}',  # YYYY.MM.DD
        r'\d{2}-\d{2}-\d{2}',  # YY-MM-DD
        r'\d{2}/\d{2}/\d{2}',  # YY/MM/DD
        r'\d{2}\.\d{2}\.\d{2}'  # YY.MM.DD
    ]

    dates = []
    for text in texts:
        description = text.description
        for pattern in date_patterns:
            matches = re.findall(pattern, description)
            for match in matches:
                try:
                    # 날짜 형식을 datetime 객체로 변환
                    if '-' in match:
                        date_obj = datetime.strptime(match, '%Y-%m-%d')
                    elif '/' in match:
                        date_obj = datetime.strptime(match, '%Y/%m/%d')
                    elif '.' in match:
                        date_obj = datetime.strptime(match, '%Y.%m.%d')
                    else:
                        # YY-MM-DD, YY/MM/DD, YY.MM.DD 형식 처리
                        if '-' in match:
                            date_obj = datetime.strptime(match, '%y-%m-%d')
                        elif '/' in match:
                            date_obj = datetime.strptime(match, '%y/%m/%d')
                        elif '.' in match:
                            date_obj = datetime.strptime(match, '%y.%m.%d')
                    dates.append(date_obj)
                except ValueError as e:
                    print(f"Date parsing error: {e}")
                    continue

    if dates:
        latest_date = max(dates)
        print(f"Extracted dates: {dates}")
        print(f"Latest date: {latest_date}")
        return latest_date.strftime('%y.%m.%d')
    return None

def save_output_to_gcs(bucket_name, destination_blob_name, output_data):
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    output_str = str(output_data)
    blob.upload_from_string(output_str)
    print(f"Output saved to gs://{bucket_name}/{destination_blob_name}")

@app.route('/process_image', methods=['POST'])
def process_image():
    data = request.json
    bucket_name = data['bucket_name']
    source_blob_name = data['source_blob_name']
    destination_file_name = 'temp_image.jpg'
    destination_blob_name = data['destination_blob_name']

    try:
        download_image_from_gcs(bucket_name, source_blob_name, destination_file_name)
        expiry_date = extract_expiry_date(destination_file_name)
        save_output_to_gcs(bucket_name, destination_blob_name, {"expiry_date": expiry_date})
        return jsonify({"status": "success", "expiry_date": expiry_date})
    except KeyError as e:
        app.logger.error(f"Key error: {e}", exc_info=True)
        return jsonify({"status": "error", "message": f"Key error: {e}"}), 400
    except Exception as e:
        app.logger.error(f"Error: {e}", exc_info=True)
        return jsonify({"status": "error", "message": f"Error: {e}"}), 500

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.219.50:5000
Press CTRL+C to quit


Blob image.jpg downloaded to temp_image.jpg.


127.0.0.1 - - [18/Jun/2024 11:40:19] "POST /process_image HTTP/1.1" 200 -


Date parsing error: time data '20.09.10' does not match format '%Y.%m.%d'
Date parsing error: time data '21.09.09' does not match format '%Y.%m.%d'
Date parsing error: time data '20.09.10' does not match format '%Y.%m.%d'
Date parsing error: time data '21.09.09' does not match format '%Y.%m.%d'
Extracted dates: [datetime.datetime(2020, 9, 10, 0, 0), datetime.datetime(2021, 9, 9, 0, 0), datetime.datetime(2020, 9, 10, 0, 0), datetime.datetime(2021, 9, 9, 0, 0)]
Latest date: 2021-09-09 00:00:00
Output saved to gs://plzbe/output.txt


# 비동기방식으로 값을 웹으로 전달 - 최종(완성)

In [1]:
from flask import Flask, request, jsonify
from google.cloud import storage, vision
from google.oauth2 import service_account
import os
import re
from datetime import datetime

app = Flask(__name__)

credentials = service_account.Credentials.from_service_account_file('api_key4.json')
vision_client = vision.ImageAnnotatorClient(credentials=credentials)
storage_client = storage.Client(credentials=credentials)

def download_image_from_gcs(bucket_name, source_blob_name, destination_file_name):
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)
    print(f"Blob {source_blob_name} downloaded to {destination_file_name}.")

def extract_expiry_date(image_path):
    with open(image_path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)
    response = vision_client.text_detection(image=image)
    texts = response.text_annotations

    if not texts:
        raise KeyError("No text found in image.")

    # 날짜 패턴 정규 표현식 (예: YYYY-MM-DD, YYYY/MM/DD, YYYY.MM.DD)
    date_patterns = [
        r'\d{4}-\d{2}-\d{2}',  # YYYY-MM-DD
        r'\d{4}/\d{2}/\d{2}',  # YYYY/MM/DD
        r'\d{4}\.\d{2}\.\d{2}',  # YYYY.MM.DD
        r'\d{2}-\d{2}-\d{2}',  # YY-MM-DD
        r'\d{2}/\d{2}/\d{2}',  # YY/MM/DD
        r'\d{2}\.\d{2}\.\d{2}'  # YY.MM.DD
    ]

    dates = []
    for text in texts:
        description = text.description
        for pattern in date_patterns:
            matches = re.findall(pattern, description)
            for match in matches:
                try:
                    # 날짜 형식을 datetime 객체로 변환
                    if '-' in match:
                        if len(match.split('-')[0]) == 4:  # YYYY-MM-DD
                            date_obj = datetime.strptime(match, '%Y-%m-%d')
                        else:  # YY-MM-DD
                            date_obj = datetime.strptime(match, '%y-%m-%d')
                            if date_obj.year < 100:
                                date_obj = date_obj.replace(year=date_obj.year + 2000)
                    elif '/' in match:
                        if len(match.split('/')[0]) == 4:  # YYYY/MM/DD
                            date_obj = datetime.strptime(match, '%Y/%m/%d')
                        else:  # YY/MM/DD
                            date_obj = datetime.strptime(match, '%y/%m/%d')
                            if date_obj.year < 100:
                                date_obj = date_obj.replace(year=date_obj.year + 2000)
                    elif '.' in match:
                        if len(match.split('.')[0]) == 4:  # YYYY.MM.DD
                            date_obj = datetime.strptime(match, '%Y.%m.%d')
                        else:  # YY.MM.DD
                            date_obj = datetime.strptime(match, '%y.%m.%d')
                            if date_obj.year < 100:
                                date_obj = date_obj.replace(year=date_obj.year + 2000)
                    dates.append(date_obj)
                except ValueError as e:
                    print(f"Date parsing error: {e}")
                    continue

    if dates:
        latest_date = max(dates)
        print(f"Extracted dates: {dates}")
        print(f"Latest date: {latest_date}")
        return latest_date.strftime('%Y-%m-%d')
    return None

def save_output_to_gcs(bucket_name, destination_blob_name, output_data):
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    output_str = str(output_data)
    blob.upload_from_string(output_str)
    print(f"Output saved to gs://{bucket_name}/{destination_blob_name}")

@app.route('/process_image', methods=['POST'])
def process_image():
    data = request.json
    bucket_name = data['bucket_name']
    source_blob_name = data['source_blob_name']
    destination_file_name = 'temp_image.jpg'
    destination_blob_name = data['destination_blob_name']

    try:
        download_image_from_gcs(bucket_name, source_blob_name, destination_file_name)
        expiry_date = extract_expiry_date(destination_file_name)
        save_output_to_gcs(bucket_name, destination_blob_name, {"expiry_date": expiry_date})
        return jsonify({"status": "success", "expiry_date": expiry_date})
    except KeyError as e:
        app.logger.error(f"Key error: {e}", exc_info=True)
        return jsonify({"status": "error", "message": f"Key error: {e}"}), 400
    except Exception as e:
        app.logger.error(f"Error: {e}", exc_info=True)
        return jsonify({"status": "error", "message": f"Error: {e}"}), 500

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.219.50:5000
Press CTRL+C to quit


Blob image.jpg downloaded to temp_image.jpg.


127.0.0.1 - - [21/Jun/2024 14:54:44] "POST /process_image HTTP/1.1" 200 -


Extracted dates: [datetime.datetime(2015, 11, 9, 0, 0), datetime.datetime(2015, 11, 9, 0, 0)]
Latest date: 2015-11-09 00:00:00
Output saved to gs://plzbe/output.txt
Blob image.jpg downloaded to temp_image.jpg.


...

Date parsing error: time data '03.02.00' does not match format '%y.%m.%d'
Date parsing error: time data '03.13.00' does not match format '%y.%m.%d'
Date parsing error: time data '03.02.00' does not match format '%y.%m.%d'
Date parsing error: time data '03.13.00' does not match format '%y.%m.%d'
Output saved to gs://plzbe/output.txt
Blob image.jpg downloaded to temp_image.jpg.


127.0.0.1 - - [21/Jun/2024 14:55:33] "POST /process_image HTTP/1.1" 200 -


Extracted dates: [datetime.datetime(2020, 9, 10, 0, 0), datetime.datetime(2021, 9, 9, 0, 0), datetime.datetime(2020, 9, 10, 0, 0), datetime.datetime(2021, 9, 9, 0, 0), datetime.datetime(2020, 9, 10, 0, 0), datetime.datetime(2020, 9, 10, 0, 0), datetime.datetime(2021, 9, 9, 0, 0), datetime.datetime(2021, 9, 9, 0, 0)]
Latest date: 2021-09-09 00:00:00
Output saved to gs://plzbe/output.txt
Blob image.jpg downloaded to temp_image.jpg.


127.0.0.1 - - [21/Jun/2024 14:55:36] "POST /process_image HTTP/1.1" 200 -


Date parsing error: time data '03.02.00' does not match format '%y.%m.%d'
Date parsing error: time data '03.13.00' does not match format '%y.%m.%d'
Date parsing error: time data '03.02.00' does not match format '%y.%m.%d'
Date parsing error: time data '03.13.00' does not match format '%y.%m.%d'
Output saved to gs://plzbe/output.txt


# MM-DD값 추가 및 예외사항 (우유 유통기한 기준) 수정

In [ ]:
from flask import Flask, request, jsonify
from google.cloud import storage, vision
from google.oauth2 import service_account
import os
import re
from datetime import datetime

app = Flask(__name__)

credentials = service_account.Credentials.from_service_account_file('api_key4.json')
vision_client = vision.ImageAnnotatorClient(credentials=credentials)
storage_client = storage.Client(credentials=credentials)

def download_image_from_gcs(bucket_name, source_blob_name, destination_file_name):
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)
    print(f"Blob {source_blob_name} downloaded to {destination_file_name}.")

def extract_expiry_date(image_path):
    with open(image_path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)
    response = vision_client.text_detection(image=image)
    texts = response.text_annotations

    if not texts:
        raise KeyError("No text found in image.")

    # 날짜 패턴 정규 표현식 (예: YYYY-MM-DD, YYYY/MM/DD, YYYY.MM.DD, MM-DD)
    date_patterns = [
        r'\d{4}-\d{2}-\d{2}',  # YYYY-MM-DD
        r'\d{4}/\d{2}/\d{2}',  # YYYY/MM/DD
        r'\d{4}\.\d{2}\.\d{2}',  # YYYY.MM.DD
        r'\d{2}-\d{2}-\d{2}',  # YY-MM-DD
        r'\d{2}/\d{2}/\d{2}',  # YY/MM/DD
        r'\d{2}\.\d{2}\.\d{2}',  # YY.MM.DD
        r'\d{2}-\d{2}',  # MM-DD
        r'\d{2}\.\d{2}'  # MM.DD
    ]

    dates = []
    current_year = datetime.now().year
    current_date = datetime.now()
    for text in texts:
        description = text.description
        for pattern in date_patterns:
            matches = re.findall(pattern, description)
            for match in matches:
                try:
                    # 날짜 형식을 datetime 객체로 변환
                    if '-' in match:
                        parts = match.split('-')
                        if len(parts) == 3 and len(parts[0]) == 4:  # YYYY-MM-DD
                            date_obj = datetime.strptime(match, '%Y-%m-%d')
                        elif len(parts) == 3:  # YY-MM-DD
                            date_obj = datetime.strptime(match, '%y-%m-%d')
                            if date_obj.year < 100:
                                if date_obj.year <= 12:  # YY가 12 이하인 경우
                                    date_obj = date_obj.replace(year=current_year)
                                else:
                                    date_obj = date_obj.replace(year=date_obj.year + 2000)
                        else:  # MM-DD
                            mm_dd_date = datetime.strptime(match, '%m-%d')
                            if mm_dd_date < datetime.strptime("01-10", '%m-%d'):
                                date_obj = datetime.strptime(f"{current_year+1}-{match}", '%Y-%m-%d')
                            else:
                                date_obj = datetime.strptime(f"{current_year}-{match}", '%Y-%m-%d')
                            if date_obj < current_date:
                                date_obj = date_obj.replace(year=date_obj.year + 1)
                    elif '/' in match:
                        parts = match.split('/')
                        if len(parts) == 3 and len(parts[0]) == 4:  # YYYY/MM/DD
                            date_obj = datetime.strptime(match, '%Y/%m/%d')
                        elif len(parts) == 3:  # YY/MM/DD
                            date_obj = datetime.strptime(match, '%y/%m/%d')
                            if date_obj.year < 100:
                                if date_obj.year <= 12:  # YY가 12 이하인 경우
                                    date_obj = date_obj.replace(year=current_year)
                                else:
                                    date_obj = date_obj.replace(year=date_obj.year + 2000)
                        else:  # MM/DD
                            mm_dd_date = datetime.strptime(match, '%m/%d')
                            if mm_dd_date < datetime.strptime("01/10", '%m/%d'):
                                date_obj = datetime.strptime(f"{current_year+1}/{match}", '%Y/%m/%d')
                            else:
                                date_obj = datetime.strptime(f"{current_year}/{match}", '%Y/%m/%d')
                            if date_obj < current_date:
                                date_obj = date_obj.replace(year=date_obj.year + 1)
                    elif '.' in match:
                        parts = match.split('.')
                        if len(parts) == 3 and len(parts[0]) == 4:  # YYYY.MM.DD
                            date_obj = datetime.strptime(match, '%Y.%m.%d')
                        elif len(parts) == 3:  # YY.MM.DD
                            date_obj = datetime.strptime(match, '%y.%m.%d')
                            if date_obj.year < 100:
                                if date_obj.year <= 12:  # YY가 12 이하인 경우
                                    date_obj = date_obj.replace(year=current_year)
                                else:
                                    date_obj = date_obj.replace(year=date_obj.year + 2000)
                        else:  # MM.DD
                            mm_dd_date = datetime.strptime(match, '%m.%d')
                            if mm_dd_date < datetime.strptime("01.10", '%m.%d'):
                                date_obj = datetime.strptime(f"{current_year+1}.{match}", '%Y.%m.%d')
                            else:
                                date_obj = datetime.strptime(f"{current_year}.{match}", '%Y.%m.%d')
                            if date_obj < current_date:
                                date_obj = date_obj.replace(year=date_obj.year + 1)
                    dates.append(date_obj)
                except ValueError as e:
                    print(f"Date parsing error: {e}")
                    continue

    if dates:
        latest_date = max(dates)
        print(f"Extracted dates: {dates}")
        print(f"Latest date: {latest_date}")
        return latest_date.strftime('%Y-%m-%d')
    return None

def save_output_to_gcs(bucket_name, destination_blob_name, output_data):
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    output_str = str(output_data)
    blob.upload_from_string(output_str)
    print(f"Output saved to gs://{bucket_name}/{destination_blob_name}")

@app.route('/process_image', methods=['POST'])
def process_image():
    data = request.json
    bucket_name = data['bucket_name']
    source_blob_name = data['source_blob_name']
    destination_file_name = 'temp_image.jpg'
    destination_blob_name = data['destination_blob_name']

    try:
        download_image_from_gcs(bucket_name, source_blob_name, destination_file_name)
        expiry_date = extract_expiry_date(destination_file_name)
        save_output_to_gcs(bucket_name, destination_blob_name, {"expiry_date": expiry_date})
        return jsonify({"status": "success", "expiry_date": expiry_date})
    except KeyError as e:
        app.logger.error(f"Key error: {e}", exc_info=True)
        return jsonify({"status": "error", "message": f"Key error: {e}"}), 400
    except Exception as e:
        app.logger.error(f"Error: {e}", exc_info=True)
        return jsonify({"status": "error", "message": f"Error: {e}"}), 500

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.219.50:5000
Press CTRL+C to quit


Blob image.jpg downloaded to temp_image.jpg.


127.0.0.1 - - [24/Jun/2024 12:15:45] "POST /process_image HTTP/1.1" 200 -


Date parsing error: time data '20.09' does not match format '%m.%d'
Date parsing error: time data '21.09' does not match format '%m.%d'
Date parsing error: time data '20.09' does not match format '%m.%d'
Date parsing error: time data '21.09' does not match format '%m.%d'
Extracted dates: [datetime.datetime(2020, 9, 10, 0, 0), datetime.datetime(2021, 9, 9, 0, 0), datetime.datetime(2020, 9, 10, 0, 0), datetime.datetime(2021, 9, 9, 0, 0), datetime.datetime(2020, 9, 10, 0, 0), datetime.datetime(2020, 9, 10, 0, 0), datetime.datetime(2021, 9, 9, 0, 0), datetime.datetime(2021, 9, 9, 0, 0)]
Latest date: 2021-09-09 00:00:00
Output saved to gs://plzbe/output.txt
Blob image.jpg downloaded to temp_image.jpg.


...

Date parsing error: time data '21.03' does not match format '%m.%d'
Date parsing error: time data '21.03' does not match format '%m.%d'
Extracted dates: [datetime.datetime(2021, 3, 29, 0, 0), datetime.datetime(2021, 3, 29, 0, 0), datetime.datetime(2021, 3, 29, 0, 0), datetime.datetime(2021, 3, 29, 0, 0)]
Latest date: 2021-03-29 00:00:00
Output saved to gs://plzbe/output.txt
Blob image.jpg downloaded to temp_image.jpg.


127.0.0.1 - - [24/Jun/2024 14:47:12] "POST /process_image HTTP/1.1" 200 -


Date parsing error: time data '25-01' does not match format '%m-%d'
Date parsing error: time data '25-01' does not match format '%m-%d'
Extracted dates: [datetime.datetime(2025, 1, 10, 0, 0), datetime.datetime(2025, 1, 10, 0, 0), datetime.datetime(2025, 1, 10, 0, 0), datetime.datetime(2025, 1, 10, 0, 0)]
Latest date: 2025-01-10 00:00:00
Output saved to gs://plzbe/output.txt
Blob image.jpg downloaded to temp_image.jpg.


127.0.0.1 - - [24/Jun/2024 14:47:14] "POST /process_image HTTP/1.1" 200 -


Extracted dates: [datetime.datetime(2025, 6, 6, 0, 0), datetime.datetime(2025, 1, 6, 0, 0), datetime.datetime(2025, 1, 6, 0, 0), datetime.datetime(2025, 1, 6, 0, 0), datetime.datetime(2025, 6, 6, 0, 0), datetime.datetime(2025, 1, 6, 0, 0)]
Latest date: 2025-06-06 00:00:00
Output saved to gs://plzbe/output.txt


# 행이 여러개일때 이미지 추가

In [ ]:
from flask import Flask, request, jsonify
from google.cloud import storage, vision
from google.oauth2 import service_account
import os
import re
from datetime import datetime

app = Flask(__name__)

credentials = service_account.Credentials.from_service_account_file('api_key4.json')
vision_client = vision.ImageAnnotatorClient(credentials=credentials)
storage_client = storage.Client(credentials=credentials)

def download_image_from_gcs(bucket_name, source_blob_name, destination_file_name):
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)
    print(f"Blob {source_blob_name} downloaded to {destination_file_name}.")

def extract_expiry_date(image_path):
    with open(image_path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)
    response = vision_client.text_detection(image=image)
    texts = response.text_annotations

    if not texts:
        raise KeyError("No text found in image.")

    # 날짜 패턴 정규 표현식 (예: YYYY-MM-DD, YYYY/MM/DD, YYYY.MM.DD, MM-DD)
    date_patterns = [
        r'\d{4}-\d{2}-\d{2}',  # YYYY-MM-DD
        r'\d{4}/\d{2}/\d{2}',  # YYYY/MM/DD
        r'\d{4}\.\d{2}\.\d{2}',  # YYYY.MM.DD
        r'\d{2}-\d{2}-\d{2}',  # YY-MM-DD
        r'\d{2}/\d{2}/\d{2}',  # YY/MM/DD
        r'\d{2}\.\d{2}\.\d{2}',  # YY.MM.DD
        r'\d{2}-\d{2}',  # MM-DD
        r'\d{2}\.\d{2}'  # MM.DD
    ]

    dates = []
    current_year = datetime.now().year
    current_date = datetime.now()
    for text in texts:
        description = text.description
        for pattern in date_patterns:
            matches = re.findall(pattern, description)
            for match in matches:
                try:
                    # 날짜 형식을 datetime 객체로 변환
                    if '-' in match:
                        parts = match.split('-')
                        if len(parts) == 3 and len(parts[0]) == 4:  # YYYY-MM-DD
                            date_obj = datetime.strptime(match, '%Y-%m-%d')
                        elif len(parts) == 3:  # YY-MM-DD
                            date_obj = datetime.strptime(match, '%y-%m-%d')
                            if date_obj.year < 100:
                                if date_obj.year <= 12:  # YY가 12 이하인 경우
                                    date_obj = date_obj.replace(year=current_year)
                                else:
                                    date_obj = date_obj.replace(year=date_obj.year + 2000)
                        else:  # MM-DD
                            mm_dd_date = datetime.strptime(match, '%m-%d')
                            if mm_dd_date < datetime.strptime("01-10", '%m-%d'):
                                date_obj = datetime.strptime(f"{current_year+1}-{match}", '%Y-%m-%d')
                            else:
                                date_obj = datetime.strptime(f"{current_year}-{match}", '%Y-%m-%d')
                            if date_obj < current_date:
                                date_obj = date_obj.replace(year=date_obj.year + 1)
                    elif '/' in match:
                        parts = match.split('/')
                        if len(parts) == 3 and len(parts[0]) == 4:  # YYYY/MM/DD
                            date_obj = datetime.strptime(match, '%Y/%m/%d')
                        elif len(parts) == 3:  # YY/MM/DD
                            date_obj = datetime.strptime(match, '%y/%m/%d')
                            if date_obj.year < 100:
                                if date_obj.year <= 12:  # YY가 12 이하인 경우
                                    date_obj = date_obj.replace(year=current_year)
                                else:
                                    date_obj = date_obj.replace(year=date_obj.year + 2000)
                        else:  # MM/DD
                            mm_dd_date = datetime.strptime(match, '%m/%d')
                            if mm_dd_date < datetime.strptime("01/10", '%m/%d'):
                                date_obj = datetime.strptime(f"{current_year+1}/{match}", '%Y/%m/%d')
                            else:
                                date_obj = datetime.strptime(f"{current_year}/{match}", '%Y/%m/%d')
                            if date_obj < current_date:
                                date_obj = date_obj.replace(year=date_obj.year + 1)
                    elif '.' in match:
                        parts = match.split('.')
                        if len(parts) == 3 and len(parts[0]) == 4:  # YYYY.MM.DD
                            date_obj = datetime.strptime(match, '%Y.%m.%d')
                        elif len(parts) == 3:  # YY.MM.DD
                            date_obj = datetime.strptime(match, '%y.%m.%d')
                            if date_obj.year < 100:
                                if date_obj.year <= 12:  # YY가 12 이하인 경우
                                    date_obj = date_obj.replace(year=current_year)
                                else:
                                    date_obj = date_obj.replace(year(date_obj.year + 2000))
                        else:  # MM.DD
                            mm_dd_date = datetime.strptime(match, '%m.%d')
                            if mm_dd_date < datetime.strptime("01.10", '%m.%d'):
                                date_obj = datetime.strptime(f"{current_year+1}.{match}", '%Y.%m.%d')
                            else:
                                date_obj = datetime.strptime(f"{current_year}.{match}", '%Y.%m.%d')
                            if date_obj < current_date:
                                date_obj = date_obj.replace(year(date_obj.year + 1))
                    dates.append(date_obj)
                except ValueError as e:
                    print(f"Date parsing error: {e}")
                    continue

    if dates:
        latest_date = max(dates)
        print(f"Extracted dates: {dates}")
        print(f"Latest date: {latest_date}")
        return latest_date.strftime('%Y-%m-%d')
    return None

def save_output_to_gcs(bucket_name, destination_blob_name, output_data):
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    output_str = str(output_data)
    blob.upload_from_string(output_str)
    print(f"Output saved to gs://{bucket_name}/{destination_blob_name}")

@app.route('/process_image', methods=['POST'])
def process_image():
    data = request.json
    bucket_name = data['bucket_name']
    source_blob_name = data['source_blob_name']
    destination_file_name = 'temp_image.jpg'
    destination_blob_name = data['destination_blob_name']

    try:
        download_image_from_gcs(bucket_name, source_blob_name, destination_file_name)
        expiry_date = extract_expiry_date(destination_file_name)
        save_output_to_gcs(bucket_name, destination_blob_name, {"expiry_date": expiry_date})
        return jsonify({"status": "success", "expiry_date": expiry_date})
    except KeyError as e:
        app.logger.error(f"Key error: {e}", exc_info=True)
        return jsonify({"status": "error", "message": f"Key error: {e}"}), 400
    except Exception as e:
        app.logger.error(f"Error: {e}", exc_info=True)
        return jsonify({"status": "error", "message": f"Error: {e}"}), 500

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.219.50:5000
Press CTRL+C to quit


Blob image.jpg downloaded to temp_image.jpg.


127.0.0.1 - - [02/Jul/2024 12:37:52] "POST /process_image HTTP/1.1" 200 -


Date parsing error: time data '21.03' does not match format '%m.%d'
Date parsing error: time data '21.03' does not match format '%m.%d'
Extracted dates: [datetime.datetime(2021, 3, 29, 0, 0), datetime.datetime(2021, 3, 29, 0, 0), datetime.datetime(2021, 3, 29, 0, 0), datetime.datetime(2021, 3, 29, 0, 0)]
Latest date: 2021-03-29 00:00:00
Output saved to gs://plzbe/output.txt
Blob image.jpg downloaded to temp_image.jpg.


...

Date parsing error: time data '24.05' does not match format '%m.%d'
Date parsing error: time data '24.05' does not match format '%m.%d'
Extracted dates: [datetime.datetime(2024, 5, 13, 0, 0), datetime.datetime(2024, 5, 13, 0, 0)]
Latest date: 2024-05-13 00:00:00
Output saved to gs://plzbe/output.txt
Blob image.jpg downloaded to temp_image.jpg.


127.0.0.1 - - [02/Jul/2024 13:23:56] "POST /process_image HTTP/1.1" 200 -


Date parsing error: time data '15.11' does not match format '%m.%d'
Date parsing error: time data '15.11' does not match format '%m.%d'
Extracted dates: [datetime.datetime(2015, 11, 9, 0, 0), datetime.datetime(2015, 11, 9, 0, 0)]
Latest date: 2015-11-09 00:00:00
Output saved to gs://plzbe/output.txt
Blob image.jpg downloaded to temp_image.jpg.


127.0.0.1 - - [02/Jul/2024 14:17:04] "POST /process_image HTTP/1.1" 200 -


Date parsing error: time data '24.05' does not match format '%m.%d'
Date parsing error: time data '24.05' does not match format '%m.%d'
Extracted dates: [datetime.datetime(2024, 5, 24, 0, 0), datetime.datetime(2024, 5, 24, 0, 0), datetime.datetime(2024, 5, 24, 0, 0), datetime.datetime(2024, 5, 24, 0, 0)]
Latest date: 2024-05-24 00:00:00
Output saved to gs://plzbe/output.txt
